# 📐 2D Lattice Basis Reduction – Interactive Notebook

This notebook demonstrates the step-by-step process of **reducing a two-dimensional lattice basis** using a simple Gram-Schmidt–style algorithm.

## 🔍 What it does:
- Takes an initial integer basis `{b1, b2}` of a 2D lattice
- Applies iterative reduction (projection & subtraction)
- Preserves the **same lattice** (structure of all integer linear combinations)
- Finds a **shorter and nearly orthogonal** basis

## 📊 Features included:
- Full reduction algorithm (`reduce_2d_basis`)
- Visualizations of the basis before and after reduction
- Randomized testing with lattice-equivalence checks
- Utility functions for table formatting and validation

## 🧠 Why it matters:
Lattice reduction is a fundamental tool in **cryptography**, **geometry of numbers**, and **computational number theory**. This notebook helps you understand how reduction works visually and numerically in 2D before moving to higher dimensions or LLL.

---

### 🔧 Function: `reduce_2d_basis(basis1, basis2)`

This function performs **basis reduction** on two 2D lattice vectors using a simple Gram-Schmidt-like algorithm.

#### 📌 Input:
- `b1`, `b2`: NumPy arrays representing the initial basis vectors (shape `(R^2)` each)

#### ✅ Output:
- A list `[steps, basis1_reduced, basis2_reduced]`, representing the reduced basis vectors, and amount of steps.

In [10]:
import numpy as np

def reduce_2d_basis(basis1, basis2):

    initial_basis = [ basis1.copy(), basis2.copy() ]

    data = []

    steps = 0

    while True:

        if np.linalg.norm(basis2) < np.linalg.norm(basis1):
            basis1, basis2 = basis2, basis1
            continue


        t = round(np.dot(basis1, basis2) / np.dot(basis1, basis1))

        data.append({
            'step': steps,
            'b1': basis1.copy(),
            'b2': basis2.copy(),
        })

        steps += 1


        if t == 0:
            break

        basis2 = basis2 - t * basis1


    return data

### 📊 2D Basis Reduction Table

This table shows the step-by-step reduction of a 2D lattice basis:

- **`step`** — the number of the reduction step
- **`b1`, `b2`** — the basis vectors at each step
- The final row labeled **`→ shortest`** indicates the shortest vector found in the final reduced basis

The goal of the reduction is to simplify the basis while preserving the original lattice:
- Making vectors shorter
- Moving them closer to orthogonality

In [9]:
import pandas as pd
from IPython.display import display

def build_basis_table(data):
    rows = []
    for entry in data:
        rows.append({
            'step': entry['step'],
            'b1': f"[{entry['b1'][0]}, {entry['b1'][1]}]",
            'b2': f"[{entry['b2'][0]}, {entry['b2'][1]}]",
        })

    last = data[-1]
    b1, b2 = last['b1'], last['b2']
    shortest = b1 if np.linalg.norm(b1) <= np.linalg.norm(b2) else b2

    rows.append({
        'step': '→ shortest',
        'b1': f"[{shortest[0]}, {shortest[1]}]",
        'b2': ''
    })

    return pd.DataFrame(rows)


b1 = np.array([31, 59])
b2 = np.array([37, 70])

data = reduce_2d_basis(b1, b2)
table = build_basis_table(data)
display(table.style.hide(axis="index"))

step,b1,b2
0,"[31, 59]","[37, 70]"
1,"[6, 11]","[31, 59]"
2,"[1, 4]","[6, 11]"
3,"[3, -1]","[1, 4]"
→ shortest,"[3, -1]",


# ✅ Lattice Reduction Test Log

**Test configuration:**

- Total tests: `5`
- Vector range: `[-50, 50]`
- Min |component|: `10`
- Algorithm: `2D basis reduction`
- Check:
  - ✅ Same lattice (via invertible integer transformation)
  - ✅ Shorter vector after reduction

In [17]:
def check_same_lattice(basis1_init, basis2_init, basis1_reduced, basis2_reduced):
    Basis_initial = np.column_stack([basis1_init, basis2_init])
    Basis_reduced = np.column_stack([basis1_reduced, basis2_reduced])

    T = np.linalg.solve(Basis_initial, Basis_reduced)

    return np.allclose(T, np.round(T)) and round(abs(np.linalg.det(T))) == 1

#reduced = [data[-1]['b1'], data[-1]['b2']]
#check_same_lattice(b1, b2, reduced[0], reduced[1])

def run_random_basis_tests(n=5, min_abs=10, max_val=50):
    passed = 0
    generated = 0

    def generate_vector():
        while True:
            v = np.random.randint(-max_val, max_val + 1, size=2)
            if all(abs(x) >= min_abs for x in v):
                return v

    while generated < n:
        b1 = generate_vector()
        b2 = generate_vector()

        if np.linalg.matrix_rank(np.column_stack([b1, b2])) < 2:
            continue

        generated += 1
        log = reduce_2d_basis(b1, b2)
        b1r, b2r = log[-1]['b1'], log[-1]['b2']

        same = check_same_lattice(b1, b2, b1r, b2r)
        original_len = min(np.linalg.norm(b1), np.linalg.norm(b2))
        reduced_len = min(np.linalg.norm(b1r), np.linalg.norm(b2r))
        improved = reduced_len <= original_len

        if same and improved:
            shortest = b1r if np.linalg.norm(b1r) <= np.linalg.norm(b2r) else b2r
            print(f"✅ Test {generated}: PASS")
            print(f"  b1 = {b1}, b2 = {b2}")
            print(f"  shortest vector: {shortest}  ||v|| = {round(np.linalg.norm(shortest), 4)}\n")
            passed += 1
        else:
            print(f"❌ Test {generated}: FAIL")
            print(f"  b1 = {b1}, b2 = {b2}")
            print(f"  reduced b1 = {b1r}, b2 = {b2r}")
            print(f"  same lattice = {same}, reduced = {improved}\n")

    print(f"\n📊 {passed}/{n} tests passed.")

run_random_basis_tests(5)

✅ Test 1: PASS
  b1 = [-42  38], b2 = [19 37]
  shortest vector: [19 37]  ||v|| = 41.5933

✅ Test 2: PASS
  b1 = [-47 -41], b2 = [46 44]
  shortest vector: [-1  3]  ||v|| = 3.1623

✅ Test 3: PASS
  b1 = [-26  24], b2 = [ 38 -26]
  shortest vector: [12 -2]  ||v|| = 12.1655

✅ Test 4: PASS
  b1 = [40 43], b2 = [ 17 -39]
  shortest vector: [ 17 -39]  ||v|| = 42.5441

✅ Test 5: PASS
  b1 = [-37 -10], b2 = [-28 -34]
  shortest vector: [  9 -24]  ||v|| = 25.632


📊 5/5 tests passed.
